# Prix des stations 

In [1]:
import urllib.request, json, requests
import pandas as pd
import numpy as np
import unicodedata
from scipy.stats import norm


from all_functions import *

### 1. Prix des stations de Métro :

In [2]:
import time
import re

In [3]:
adresse = "51, rue de la harpe Paris"

On enregistre le prix des stations de métro à moins de 1h du centre de Paris (chez JayZer)

In [4]:
def get_station_price(coordonnees, temps_max=3600) :
    navitia_url = str('https://api.navitia.io/v1/coverage/fr-idf/journeys?from='
                      +str(coordonnees[0])+';'+str(coordonnees[1])
                      +'&max_duration='+str(temps_max))#+"&allowed_id[]=physical_mode:Metro&allowed_id[]=physical_mode:RER"
    navitia_token = '4e8a1312-13a2-4fb8-9e98-aa9cd51b6a11'
    time_json = requests.get(navitia_url, headers={'Authorization': navitia_token})
    time = ''
    for line in time_json :
        time += line.decode().strip()
    dictio = json.loads(time)
    
    name_station, price, coord_station, long, lat = [[] for i in range(5)]
    
    for i in range(len(dictio["journeys"])):
        name_station.append(dictio["journeys"][i]["to"]["name"])
        long.append(dictio["journeys"][i]["to"]["stop_point"]["coord"]["lon"])
        lat.append(dictio["journeys"][i]["to"]["stop_point"]["coord"]["lat"])
        price.append(get_price([long[i], lat[i]]))
        
    coord_station = [invert_coord(coord) for coord in coord_station]
        
    df = pd.DataFrame(list(zip(name_station, price, long, lat)),
                      columns=["name_station", "price", "long", "lat"])
    return(df)

<div class="alert alert-info">
Le fameux test

In [5]:
start = time.time()

In [6]:
adresse = "51, rue de la harpe Paris"
coord = get_coordonnees(adresse)
df = get_station_price(coord)

KeyboardInterrupt: 

In [ ]:
print(time.time() - start)

In [ ]:
df = df.drop_duplicates(subset="name_station").reset_index(drop=True)

In [ ]:
df

In [ ]:
#C'était pour enlever les "(Paris)" et les "(Créteil)", mais inutile en fait
#df["name_station"] = df["name_station"].apply(lambda x: re.sub(r"\(.*\)", "", x))

In [ ]:
df = prices.copy()

In [ ]:
df.to_pickle("station_price.pkl")

# Load from pkl

In [8]:
prices = pd.read_pickle('data/station_price_full.pkl')
prices

,name_station,price,long,lat
0,Maubert-Mutualité (Paris),11196.177547,2.348926,48.849717
1,Cité (Paris),13998.734033,2.346203,48.855685
2,Les Écoles (Paris),11939.299414,2.342103,48.849789
3,Saint-Michel - Saint-Germain (Paris),11202.852386,2.344296,48.853483
4,Dante (Paris),11111.821641,2.345984,48.850598
...,...,...,...,...
5324,Église (Saint-Mard),2938.777037,2.691614,49.036558
5325,Avenue de la Gare (Saint-Mard),2654.179736,2.69616,49.033893
5326,Paul Langevin (Garges-lès-Gonesse),2337.620941,2.411453,48.977373
5327,Université Paris 8 (Saint-Denis),9578.936953,2.365061,48.94638


In [ ]:
def get_station(coordonnees, temps_max=500) :
    navitia_url = str('https://api.navitia.io/v1/coverage/fr-idf/journeys?from='
                      +str(coordonnees[0])+';'+str(coordonnees[1])
                      +'&max_duration='+str(temps_max))+"&allowed_id[]=physical_mode:Metro&allowed_id[]=physical_mode:RER"
    navitia_token = '4e8a1312-13a2-4fb8-9e98-aa9cd51b6a11'
    time_json = requests.get(navitia_url, headers={'Authorization': navitia_token})
    time = ''
    for line in time_json :
        time += line.decode().strip()
    dictio = json.loads(time)
    
    name_station, duration, price, coord_station, long, lat = [[] for i in range(6)]
    
    for i in range(len(dictio["journeys"])):
        name_station.append(dictio["journeys"][i]["to"]["name"])
        duration.append(round(dictio["journeys"][i]["duration"]/60, 1))
        long.append(dictio["journeys"][i]["to"]["stop_point"]["coord"]["lon"])
        lat.append(dictio["journeys"][i]["to"]["stop_point"]["coord"]["lat"])
        
        if name_station[-1] in prices.name_station.tolist():
            price.append(prices[prices["name_station"]== name_station[-1]].iloc[0]['price'])
        else : 
            price.append(np.nan)
        
    coord_station = [invert_coord(coord) for coord in coord_station]
        
    df = pd.DataFrame(list(zip(name_station, duration, price, long, lat)),
                      columns=["name_station", "duration", "price", "long", "lat"])
    return(df)

<div class="alert alert-info">
Le fameux test

In [ ]:
start = time.time()

In [ ]:
adresse = "51, rue de la harpe Paris"
coord = get_coordonnees(adresse)
df = get_station(coord)

In [ ]:
print(time.time() - start)

In [ ]:
df.head()